In [1]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': '../imdb_extracted/train_imdb.csv', 'val' : '../imdb_extracted/val_imdb.csv', 'test': '../imdb_extracted/test_imdb.csv'})

Using custom data configuration default-b149e5877dafa97b
Reusing dataset csv (C:\Users\AR88750\.cache\huggingface\datasets\csv\default-b149e5877dafa97b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
train_ds, val_ds, test_ds = dataset["train"], dataset["val"], dataset["test"]

In [3]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [4]:
val_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [5]:
test_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [6]:
from fastai.text.all import *
set_seed(42,True)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length = 512)

tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-b149e5877dafa97b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-5e253df6cf36d529.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-b149e5877dafa97b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-3c6590e422dc2d12.arrow


  0%|          | 0/25 [00:00<?, ?ba/s]

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments("trainer", evaluation_strategy="epoch", num_train_epochs = 4)

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
import torch

torch._C._cuda_getDeviceCount()

1

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 20000
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.306500,0.215361,0.922600
2,0.187000,0.334800,0.929600
3,0.094500,0.338443,0.933200
4,0.038400,0.380663,0.936800


Saving model checkpoint to trainer\checkpoint-500
Configuration saved in trainer\checkpoint-500\config.json
Model weights saved in trainer\checkpoint-500\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-1000
Configuration saved in trainer\checkpoint-1000\config.json
Model weights saved in trainer\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-1500
Configuration saved in trainer\checkpoint-1500\config.json
Model weights saved in trainer\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-2000
Configuration saved in trainer\checkpoint-2000\config.json
Model weights saved in trainer\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-2500
Configuration saved in trainer\checkpoint-2500\config.json
Model weights saved in trainer\checkpoint-2500\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and

TrainOutput(global_step=10000, training_loss=0.16630198612213135, metrics={'train_runtime': 11288.4244, 'train_samples_per_second': 7.087, 'train_steps_per_second': 0.886, 'total_flos': 2.279042236416e+16, 'train_loss': 0.16630198612213135, 'epoch': 4.0})

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8


{'eval_loss': 0.33571791648864746,
 'eval_accuracy': 0.94324,
 'eval_runtime': 1593.9176,
 'eval_samples_per_second': 15.685,
 'eval_steps_per_second': 1.961}

In [16]:
training_args = TrainingArguments("trainer_part2", evaluation_strategy="epoch", num_train_epochs = 1)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Epoch,Training Loss,Validation Loss,Accuracy
1,0.047300,0.364706,0.933600


Saving model checkpoint to trainer_part2\checkpoint-500
Configuration saved in trainer_part2\checkpoint-500\config.json
Model weights saved in trainer_part2\checkpoint-500\pytorch_model.bin
Saving model checkpoint to trainer_part2\checkpoint-1000
Configuration saved in trainer_part2\checkpoint-1000\config.json
Model weights saved in trainer_part2\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to trainer_part2\checkpoint-1500
Configuration saved in trainer_part2\checkpoint-1500\config.json
Model weights saved in trainer_part2\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to trainer_part2\checkpoint-2000
Configuration saved in trainer_part2\checkpoint-2000\config.json
Model weights saved in trainer_part2\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to trainer_part2\checkpoint-2500
Configuration saved in trainer_part2\checkpoint-2500\config.json
Model weights saved in trainer_part2\checkpoint-2500\pytorch_model.bin
The following columns in the evaluatio

TrainOutput(global_step=2500, training_loss=0.08262122192382812, metrics={'train_runtime': 2727.1064, 'train_samples_per_second': 7.334, 'train_steps_per_second': 0.917, 'total_flos': 5697605591040000.0, 'train_loss': 0.08262122192382812, 'epoch': 1.0})

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8


{'eval_loss': 0.3285558223724365,
 'eval_accuracy': 0.94024,
 'eval_runtime': 1572.837,
 'eval_samples_per_second': 15.895,
 'eval_steps_per_second': 1.987}

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 8


{'eval_loss': 0.2759050726890564,
 'eval_accuracy': 0.93935,
 'eval_runtime': 1279.7772,
 'eval_samples_per_second': 15.628,
 'eval_steps_per_second': 1.953}

In [13]:
model

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 25000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9375


Step,Training Loss
500,0.384200
1000,0.360500
1500,0.339900
2000,0.346800


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

In [ ]:
model.save_pretrained("xlnet_imdb_model")